In [14]:
from langgraph.graph import StateGraph,START , END
from typing import Annotated
from typing_extensions import TypedDict
from langgraph.graph.message  import add_messages
from langgraph.prebuilt import ToolNode,tools_condition
from langgraph.types import Command,interrupt  # for human assistance
from langchain.tools import tool

In [27]:
from langgraph.checkpoint.memory import MemorySaver

memory = MemorySaver()

In [9]:
from dotenv import load_dotenv
import os
load_dotenv()
os.getenv

<function os.getenv(key, default=None)>

In [12]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model="gpt-4o",)

In [15]:
from langchain_tavily import TavilySearch

@tool
def human_assistance(query: str) -> str:
    """Request assistance from a human."""
    human_response = interrupt({"query": query})
    return human_response["data"]


tavily_tool = TavilySearch()
tools = [tavily_tool, human_assistance]

llm_with_tools = llm.bind_tools(tools)

In [16]:
class State(TypedDict):
    messages : Annotated[list,add_messages]

In [28]:
graph_builder = StateGraph(State)

def Biva(State):
    """Biva is a multi-agent system that can search the web and ask for human assistance."""
    return {"messages" : [llm_with_tools.invoke(State["messages"])]}

graph_builder.add_node("Biva",Biva)
graph_builder.add_node("tools", ToolNode(tools))
graph_builder.add_edge(START, "Biva")
graph_builder.add_conditional_edges("Biva",tools_condition)
graph_builder.add_edge("tools","Biva")
graph_builder.add_edge("Biva", END)
graph = graph_builder.compile(checkpointer=memory)

In [30]:
run_cfg = {"configurable": {"thread_id": "1"}}

messages = graph.invoke(
    {"messages": [{"role": "user", "content": "I need some expert guidance for building an AI agent. Could you request assistance for me?"}]},
    config=run_cfg
)
for m in messages["messages"]:
    m.pretty_print()

================================ Human Message =================================

I need some expert guidance for building an AI agent. Could you request assistance for me?
================================== Ai Message ==================================
Tool Calls:
  human_assistance (call_CGzop4ajRwUFgYuVqVRbNOGg)
 Call ID: call_CGzop4ajRwUFgYuVqVRbNOGg
  Args:
    query: I need expert guidance for building an AI agent.


In [32]:
human_response = (
    "We, the experts are here to help! We'd recommend you check out LangGraph to build your agent."
    " It's much more reliable and extensible than simple autonomous agents."
)

human_command = Command(resume={"data": human_response})

events = graph.stream(human_command, run_cfg, stream_mode="values")
for event in events:
    if "messages" in event:
        event["messages"][-1].pretty_print()

================================== Ai Message ==================================
Tool Calls:
  human_assistance (call_CGzop4ajRwUFgYuVqVRbNOGg)
 Call ID: call_CGzop4ajRwUFgYuVqVRbNOGg
  Args:
    query: I need expert guidance for building an AI agent.
================================= Tool Message =================================
Name: human_assistance

We, the experts are here to help! We'd recommend you check out LangGraph to build your agent. It's much more reliable and extensible than simple autonomous agents.
================================== Ai Message ==================================

One of our experts recommends using **LangGraph** for building your AI agent, as it offers more reliability and extensibility compared to simple autonomous agents. Feel free to reach out if you need more details!
